# Numerical analysis of ergodicity of 1D potentials thermostatted using the DD algorithm

## Harmonic Oscillator

In [1]:
using HDF5
using StatsBase
using PyPlot

In [2]:
potentialname = "HO"
file = h5open("../poincaredata/$potentialname/b4dEHO.hdf5","r")
data = read(file, "tx");

In [3]:
t = data[:,1]
q = data[:,2]
p = data[:,3]
z = data[:,4];
Q = read(attrs(file)["Q"])
T = read(attrs(file)["T"])
beta = 1./T;

In [4]:
if potentialname == "HO"
    potential(x) = x^2/2.
elseif potentialname == "QP"
    potential(x) = x^4/4.
elseif potentialname == "MH"
    potential(x) = -1/2.*x^2 + 1/4.*x^4
end

potential (generic function with 1 method)

In [16]:
freq_exp = fit(Histogram, (q,p,z), nbins = 100);

In [17]:
edges = freq_exp.edges

(-4.3:0.1:4.2,-5.0:0.1:4.8,-1.15:0.05:1.3)

In [18]:
hist_exp = freq_exp.weights/(sum(freq_exp.weights));

In [19]:
step_x = step(edges[1])
step_y = step(edges[2])
step_z = step(edges[3])

0.05

In [20]:
qrange = edges[1][1]+step_x/2.:step_x:edges[1][end]-step_x/2.

-4.25:0.1:4.15

In [21]:
prange = edges[2][1]+step_y/2.:step_y:edges[2][end]-step_y/2.

-4.95:0.1:4.75

In [28]:
zrange = edges[3][1]+step_z/2.:step_z:edges[3][end]-step_z/2.

-1.125:0.05:1.2750000000000004

In [29]:
function jointdistribution(p::Float64,q::Float64, z::Float64)
    rhoq =  sqrt(beta/(2.*pi))*exp(-beta*potential(q))
    rhop = sqrt(beta/(2.*pi))*exp(-beta*p.^2/2.)
    rhoz = exp(z/Q)./(Q*(1.+ exp(z/Q)).^2);
    rhoq*rhop*rhoz
end
    

jointdistribution (generic function with 1 method)

In [31]:
hist_theor = Float64[jointdistribution(x,y,z) for x in qrange, y in prange, z in zrange];

In [34]:
function hellinger{T}(hist1::Array{T,3}, hist2::Array{T,3})
    distance1 = sum((sqrt(hist_exp) - sqrt(hist_theor)).^2)
    distance2 = 1./sqrt(2.)*sqrt(distance1)
end
    

hellinger (generic function with 2 methods)

In [35]:
hellinger(hist_exp, hist_theor)

30.928224230788512